In [4]:
#!/usr/bin/env python
import time
from datetime import datetime
import sys
import getpass
import argparse
import json
import os
import requests
from requests.auth import HTTPBasicAuth
from jinja2 import Template
import urllib3
import  openpyxl
import json
from  datetime  import date
from mapping import  control_vulnerability_description, security_control_number, office_org, security_checks, \
resources_required, \
scheduled_completion_date, \
milestone_with_completion_dates, \
milestone_changes, \
source_identifying_vulnerability, \
status, \
comments, \
raw_severity, \
devices_affected, \
mitigations_inhouse, \
predisposing_conditions, \
severity, \
relevance_of_threat, \
threat_description, \
likelihood, \
impact, \
impact_description, \
residual_risk_level, \
recommendations, \
resulting_residual_risk_after_proposed_mitigations \

urllib3.disable_warnings()



In [5]:
class imgRequestError(Exception):
    pass

In [12]:
def import_poam_template_xlsx(file):
    
    from openpyxl import load_workbook
    workbook = load_workbook(filename=file)
    return workbook

poam = import_poam_template_xlsx("POAM_Export_Sample.xlsx")
sheet = poam.active

/Users/jonathanspigler/opt/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:296: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [31]:
def control_vulnerability_description(vulnerability):
    
    description = vulnerability['description']
    return  description

def security_control_number():
    pass
def office_org():
    
    pass
def security_checks():
    pass
def resources_required():
    pass
def scheduled_completion_date():
    pass
def milestone_with_completion_dates():
    pass
def milestone_changes():
    pass
def source_identifying_vulnerability():
    pass
def status():
    pass
def comments():
    pass
def raw_severity():
    pass
def devices_affected():
    pass
def mitigations_inhouse():
    pass
def predisposing_conditions():
    pass
def severity():
    pass
def relevance_of_threat():
    pass
def threat_description():
    pass
def likelihood():
    pass
def impact():
    pass
def impact_description():
    pass
def residual_risk_level():
    pass
def recommendations():
    pass
def resulting_residual_risk_after_proposed_mitigations():
    pass


def populate_poam_template_xlsx(poam,images):
    
    row = 8
    sheet = poam.active
    count = 0
    
    for image in images:
        
        office_org = office_org(image)
        
        for vulnerability in image['vulnerabilities']:
            
#           Vulnerability Description
            description = control_vulnerability_description(vulnerability)

    
            
            row +=1
    
    return poam

populate_poam_template_xlsx(poam,images)



A weakness was found in postgresql-jdbc before version 42.2.5. It was possible to provide an SSL Factory and not check the host name if a host name verifier was not provided to the driver. This could lead to a condition where a man-in-the-middle attacker could masquerade as a trusted server by providing a certificate for the wrong host, as long as it was signed by a trusted CA.
In all versions of cpio before 2.13 does not properly validate input files when generating TAR archives. When cpio is used to create TAR archives from paths an attacker can write to, the resulting archive may contain files with permissions the attacker did not have or in paths he did not have access to. Extracting those archives from a high-privilege user without carefully reviewing them may lead to the compromise of the system.
An issue was discovered in disable_priv_mode in shell.c in GNU Bash through 5.0 patch 11. By default, if Bash is run with its effective UID not equal to its real UID, it will drop privil

In [30]:

image = images[0]['vulnerabilities'][0]
image['description']


'A weakness was found in postgresql-jdbc before version 42.2.5. It was possible to provide an SSL Factory and not check the host name if a host name verifier was not provided to the driver. This could lead to a condition where a man-in-the-middle attacker could masquerade as a trusted server by providing a certificate for the wrong host, as long as it was signed by a trusted CA.'

In [45]:
def output_poam_xlsx(poam,app,build):

    build = str(build)
    today = date.today()
    filename = "POAM-"+app+"-build-"+build+"-"+today.isoformat()+".xlsx"

    poam.save(filename=filename)
    
output_poam_xlsx(new_poam,"nodetester",1)

In [9]:
def parse_args():
    """
    CLI argument handling
    """

    desc = 'Generate an HTML report of CVEs per image, displaying the data to STDOUT\n'

    epilog = 'The console and user arguments can be supplied using the environment variables TL_CONSOLE and TL_USER.'
    epilog += ' The password can be passed using the environment variable TL_PASS.'
    epilog += ' The user will be prompted for the password when the TL_PASS variable is not set.'
    epilog += ' Environment variables override CLI arguments.'

    p = argparse.ArgumentParser(description=desc,epilog=epilog)
    p.add_argument('-c','--console',metavar='TL_CONSOLE', help='query the API of this Console')
    p.add_argument('-u','--user',metavar='TL_USER',help='Console username')
    p.add_argument('-p','--password',metavar='TL_PASS',help='Console user password')
    p.add_argument('-d','--debug',help='Provide a debug console dump of HTML report',action='store_true')
    # p.add_argument('-t','--target',metavar='TL_TARGET',help='Targeted entity to generate report on (e.g. container image, host, running containers) Options\
    # running_container,image,host ')
    args = p.parse_args()

    # Populate args by env vars if they're set
    envvar_map = {'TL_USER':'user','TL_CONSOLE':'console','TL_PASS':'password'}
    for evar in envvar_map.keys():
        evar_val = os.environ.get(evar,None)
        if evar_val is not None:
            setattr(args,envvar_map[evar],evar_val)

    arg_errs = []
    if len(arg_errs) > 0:
        err_msg = 'Missing argument(s): {}'.format(', '.join(arg_errs))
        p.error(err_msg)

    if getattr(args,'console',None) is None:
        args.console = raw_input('Enter console url: ')
    else:
        arg_errs.append('console (-c,--console)')
    if getattr(args,'user',None) is None:
        args.user = raw_input('Enter username: ')
    else:
        arg_errs.append('user (-u,--user)')

    # if getattr(args,'target',None) is None:
    #     args.tar = raw_input('Target resource generating reporton: ')
    # else:
    #     arg_errs.append('password (-t, --target)')

    if getattr(args,'password',None) is None:
        args.password = getpass.getpass('Enter password: ')
    else:
        arg_errs.append('password (-p, --password)')

    return args


In [10]:
def get_images_json(console,user,password,image):
    api_endpt = '/api/v1/images?'+image
    request_url = console + api_endpt
    image_req = requests.get(request_url, auth=HTTPBasicAuth(user,password), verify=False)
    if image_req.status_code != 200:
        # This means something went wrong.
        raise imgRequestError('GET /api/v1/images {} {}'.format(image_req.status_code,image_req.reason))
    return image_req.json()

In [11]:
images = get_images_json('https://twistlock-console.oceast.cloudmegalodon.us','jonathan@clearshark.com','clearshark123!','')

# for vulnerability in images[0]['vulnerabilities']:
#     print(vulnerability['cve'])
#     print(vulnerability['id'])
#     print(vulnerability['description'])

In [80]:
# for image in images:
    
#     print("Image_id:  "+ image['_id'] + "\n" + "-"*len("Image_id:  "+image['_id']))
          
#     for binary in image['startupBinaries']:
#         print(binary['name'])
        
#     print("\n")

Image_id:  sha256:98cae52681f43cc1c330555f348cab1813fa4142ce878b0c07b26c5c2eaecba3
----------------------------------------------------------------------------------
bash
watch
split
ls
echo
curl
trust
openssl
tr
head
csplit
java
hostname
mkdir
rm
date
keytool
cat
vi
sed
python
basename
false
sleep
dirname
expr
grep
mv
pwd
readlink
gawk
kill
uname


Image_id:  sha256:6b2f02b228ba457c437018987d5552c95c7c7a4f720a64a1b339ab573691444d
----------------------------------------------------------------------------------
bash
cut
echo
false
keytool
head
csplit
mkdir
cat
sed
tr
openssl
trust
cp
split
ls
rm
which
col
users
size
tail
host
timeout
mv
column
w
groups
python
login
locale
raw
expand
vi
more
ownership
install
tty
dir
seq
dig
paste
sleep
ldconfig
grep
dirname
java
printf
getopt
find
yes
uname


Image_id:  sha256:752ded5721543a8bf83dce1da2556876cbbdf9fc105b4dcd999ef6837b45c30d
----------------------------------------------------------------------------------
rm
less
echo
env
sort
cp
tr
w